In [2]:
import wmi
import gc
wmic = wmi.WMI()
process = wmic.Win32_Process(name="cloudmusic.exe")
process = [p for p in process if '--type=' not in p.ole_object.CommandLine]
if not process:
    raise RuntimeError('No candidate process found')
elif len(process) != 1:
    raise RuntimeError('Multiple candidate processes found!')
else:
    process = process[0]
    pid = process.ole_object.ProcessId
    del process
    del wmic
    gc.collect()

In [5]:
from pyMeow import open_process, get_module, r_float64, close_process, get_module, r_bytes, r_uint
import psutil

process = open_process(pid)
ps_util_process = psutil.Process(pid)
base_address = get_module(process, 'cloudmusic.dll')['base']

In [ ]:
for offset in range(0xA00000, 0xC00000): # Change to estimated memory range
    try:
        songid_array = r_uint(process, base_address + offset)
        song_id = r_bytes(process, songid_array, 0x14).decode('utf-16')
        if song_id == '1458767558':  # Change to current song ID
            print(f'found song array offset at {hex(offset)}: {song_id}')
    except Exception as e:
        pass

    try:
        length = r_float64(process, base_address + offset)
        if length > 255 and length < 256:  # Change to song length, in seconds
            print(f'found length at {hex(offset)}: {length}')
    except Exception as e:
        pass

    try:
        current = r_float64(process, base_address + offset)
        if current > 145 and current < 146:  # Change to current progress, in seconds
            print(f'found current at {hex(offset)}: {current}')
    except Exception as e:
        pass

In [4]:
hex(base_address + 0xb15654)

'0x3ac5654'

In [5]:
import ctypes
from ctypes import wintypes

user32 = ctypes.windll.user32

WNDENUMPROC = ctypes.WINFUNCTYPE(
    wintypes.BOOL,
    wintypes.HWND,    # _In_ hWnd
    wintypes.LPARAM,) # _In_ lParam

user32.EnumWindows.argtypes = (
   WNDENUMPROC,      # _In_ lpEnumFunc
   wintypes.LPARAM,) # _In_ lParam

user32.IsWindowVisible.argtypes = (
    wintypes.HWND,) # _In_ hWnd

user32.GetWindowThreadProcessId.restype = wintypes.DWORD
user32.GetWindowThreadProcessId.argtypes = (
  wintypes.HWND,     # _In_      hWnd
  wintypes.LPDWORD,) # _Out_opt_ lpdwProcessId

user32.GetWindowTextLengthW.argtypes = (
   wintypes.HWND,) # _In_ hWnd

user32.GetWindowTextW.argtypes = (
    wintypes.HWND,   # _In_  hWnd
    wintypes.LPWSTR, # _Out_ lpString
    ctypes.c_int,)   # _In_  nMaxCount

def get_title(pid) -> str:
    ret = ''
    @WNDENUMPROC
    def enum_proc(hWnd, lParam):
        nonlocal ret
        if user32.IsWindowVisible(hWnd):
            _pid = wintypes.DWORD()
            user32.GetWindowThreadProcessId(hWnd, ctypes.byref(_pid))
            length = user32.GetWindowTextLengthW(hWnd) + 1
            title = ctypes.create_unicode_buffer(length)
            user32.GetWindowTextW(hWnd, title, length)
            if _pid.value == pid: ret = title.value
        return True
    user32.EnumWindows(enum_proc, 0)
    return ret

get_title(pid)

'星辰大海 - 黄霄雲'